# ADM Homework 5 - Group 33
Flaminia Matteucci, Francesca Porcu, Michele Fortunato, Pol Ribó León  

## Homework 5 - Visit the Wikipedia hyperlinks graph!

In this assignment we perform an analysis of the Wikipedia Hyperlink graph. In particular, given extra information about the categories to which an article belongs to, we are curious to rank the articles according to some criteria.

For this purpose we use the Wikipedia graph released by the SNAP group.

In [1]:
# Prepare the environment
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook

Let's start importing the files

In [2]:
data_link = pd.read_csv(r'data\wiki-topcats-reduced.txt',sep="\t", header=None,engine='python' )   
data_link.columns=['start','end']
data_link.head()

,start,end
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


In [3]:
data_categories = pd.read_csv(r'data\wiki-topcats-categories.txt.gz',sep='\n')
data_categories.columns = ['Categories']
data_categories.head()

,Categories
0,Category:People_from_Worcester; 1056 1057 1058...
1,Category:Skin_conditions_resulting_from_physic...
2,Category:Visual_kei_bands; 1297 1300 1311 1312...
3,Category:Japanese_rock_music_groups; 1297 1300...
4,Category:Laboulbeniomycetes; 1941 1942 1943 19...


We need to select only the categories with more than 3500 articles.

In [4]:
l1 = []
l2 = []
for row in data_categories['Categories']:
    a = [int(s) for s in row.split() if s.isdigit()]
    l1.append(a)
    b = ''.join([i for i in row.split() if not i.isdigit()]).replace(";", "")
    l2.append(b)

In [5]:
df_articles = pd.DataFrame({'List of articles':l1})
df_categories = pd.DataFrame(l2, columns = ['Categories'])

df_full = df_articles.combine_first(df_categories)

df_full.head()

,Categories,List of articles
0,Category:People_from_Worcester,"[1056, 1057, 1058, 1059, 1060, 60971, 76515, 7..."
1,Category:Skin_conditions_resulting_from_physic...,"[971, 973, 1166, 1167, 1168, 1169, 1170, 1171,..."
2,Category:Visual_kei_bands,"[1297, 1300, 1311, 1312, 1313, 1314, 1315, 131..."
3,Category:Japanese_rock_music_groups,"[1297, 1300, 1313, 1314, 1315, 1316, 1319, 132..."
4,Category:Laboulbeniomycetes,"[1941, 1942, 1943, 1944, 1945, 1947, 1948, 194..."


In [33]:
l = []
for item in df_full['List of articles']:
    l.append(len(item))
    
df_full['Length'] = pd.Series(l)

df_full = df_full[df_full.Length > 3500] # Remove items with lenght < 3500

df_full.index = pd.np.arange(0,len(df_full)) # Reindex

df_full

,Categories,List of articles,Length
0,Category:English_footballers,"[22860, 28411, 28961, 28979, 29264, 29573, 295...",9237
1,Category:The_Football_League_players,"[14003, 23536, 27109, 27348, 27459, 27989, 280...",9467
2,Category:Association_football_forwards,"[26876, 26877, 26879, 26887, 26892, 26904, 269...",6959
3,Category:Association_football_goalkeepers,"[26900, 26909, 26917, 26960, 26966, 26984, 270...",3997
4,Category:Association_football_midfielders,"[14003, 15291, 23536, 26880, 26882, 26885, 268...",8270
5,Category:Association_football_defenders,"[15217, 22860, 26873, 26878, 26881, 26898, 269...",6668
6,Category:Living_people,"[52, 62, 64, 66, 70, 74, 79, 96, 103, 104, 105...",418223
7,Category:Year_of_birth_unknown,"[3335, 10527, 15076, 16310, 22286, 23468, 2346...",3760
8,Category:Harvard_University_alumni,"[77, 1013, 1271, 1663, 1779, 1843, 2212, 3193,...",6154
9,Category:Major_League_Baseball_pitchers,"[79, 24213, 33054, 37167, 53973, 63107, 69823,...",6580


In [7]:
dict_df_full = df_full['List of articles'].to_dict()
dict_df_full

{0: [22860,
  28411,
  28961,
  28979,
  29264,
  29573,
  29582,
  30896,
  30900,
  31902,
  33973,
  34042,
  41141,
  43573,
  48582,
  48583,
  48714,
  48718,
  48730,
  48922,
  54184,
  72481,
  72482,
  72483,
  72488,
  72493,
  72496,
  72518,
  72521,
  72522,
  72526,
  72528,
  72529,
  72530,
  72532,
  72534,
  72535,
  72536,
  72538,
  72539,
  72545,
  72546,
  72549,
  72555,
  72563,
  72567,
  72569,
  72579,
  72580,
  72583,
  72587,
  72589,
  72590,
  72591,
  72592,
  72593,
  72595,
  72605,
  72607,
  72609,
  72613,
  72616,
  72617,
  72620,
  72622,
  72628,
  72629,
  72630,
  72631,
  72633,
  72652,
  72654,
  72655,
  72656,
  72657,
  72658,
  72660,
  72661,
  72663,
  72664,
  72665,
  72668,
  72680,
  72681,
  72683,
  72685,
  72694,
  72695,
  72696,
  72698,
  72700,
  72701,
  72702,
  72703,
  72705,
  72706,
  72708,
  72709,
  72710,
  72711,
  72712,
  72713,
  72714,
  72715,
  72716,
  72717,
  72718,
  72727,
  72728,
  72735,
  72736

In [8]:
data_page_names = pd.read_table(r'data\wiki-topcats-page-names.txt.gz',compression='gzip',header=None , sep='\n')
data_page_names.head()

,0
0,0 Chiasmal syndrome
1,1 Kleroterion
2,2 Pinakion
3,3 LyndonHochschildSerre spectral sequence
4,4 Zariski's main theorem


In [9]:
#creating graph 
G=nx.from_pandas_edgelist(data_link, 'start','end' , create_using=nx.DiGraph())

In [10]:
G.is_directed()

True

In [11]:
G.number_of_nodes()

461193

In [12]:
G.number_of_edges()

2645247

Definition of BFS function

In [45]:
import collections


def breadth_first_search(graph, root): 
    
    pbar = tqdm(total = 428957, position=1)
    distance = 0
    pbar.set_description("Starting node: " + str(root))
    pbar.refresh() # to show immediately the update
    
    visited, queue = [], collections.deque([root])
    while queue:
        vertex = queue.popleft()
        try:
            for neighbour in graph[vertex]: 
                if neighbour not in visited: 
                    with open('data\distances.txt', 'a') as the_file:
                        the_file.write(str(root) + ',' + str(neighbour) + ',' + str(distance) + '\n')
                    queue.append(neighbour)
                    visited.append(neighbour)
            pbar.update(1)
            distance += 1
        except KeyError:
            continue
            
    return visited

We need to complete the data_link dataframe adding the rows referring to nodes that don't have any edge

In [15]:
missing = data_link[~data_link['end'].isin(data_link['start'])]
missing['set'] = "None"

missing = missing.drop(columns=['start'])

missing.columns = ['start','end']

missing.head()

C:\Users\mikyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,start,end
7,1354553,None
53,136,None
94,645690,None
199,114662,None
238,1035,None


In [16]:
data_link_full = data_link.append(missing)
data_link_full

,start,end
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659
5,62,279122
6,62,1089199
7,62,1354553
8,62,1400636
9,62,1403619


We can now create the adiacency list.

In order to speed up the calculations, let's filter the data link removing all the edges not belonging to the selected categories.

First, we create a set with all the ids we need.

In [40]:
id_list = []

for articles_list in df_full['List of articles']:
    id_list = id_list + articles_list
    
id_list = list(set(id_list))

id_list

[1048576,
 1048577,
 1048578,
 1048579,
 1048582,
 1048583,
 1048584,
 1048585,
 1048586,
 1048587,
 1048588,
 1048589,
 1048590,
 1048592,
 1048594,
 1048596,
 1048599,
 1048601,
 1048603,
 1048608,
 1048610,
 1048615,
 1048618,
 1048620,
 1048621,
 1048624,
 1048627,
 52,
 1048628,
 1048629,
 1048630,
 1048632,
 1048633,
 55,
 1048631,
 1048636,
 1048637,
 62,
 1048639,
 64,
 1048641,
 66,
 1048642,
 1048643,
 1048645,
 70,
 1048647,
 1048644,
 1048649,
 74,
 1048650,
 1048651,
 77,
 1048654,
 79,
 1048653,
 1048656,
 1048658,
 1048659,
 1048660,
 1048661,
 1048662,
 1048663,
 1048670,
 95,
 96,
 1048672,
 1048674,
 1048675,
 98,
 1048677,
 103,
 104,
 105,
 1048680,
 107,
 1048683,
 108,
 1048686,
 1048688,
 1048689,
 112,
 113,
 117,
 118,
 119,
 1048695,
 121,
 122,
 1048700,
 126,
 1048703,
 1048706,
 133,
 1048709,
 1458915,
 136,
 134,
 1048713,
 1048715,
 140,
 1458916,
 1048710,
 137,
 1048720,
 145,
 1048721,
 1048722,
 1048723,
 1048725,
 147,
 152,
 153,
 154,
 155,
 156,


Now we can filter the data_link. We can exclude edges where 'start' and 'end' are not present in the id_list

In [41]:
data_link_filtered = data_link_full[
    data_link_full['start'].isin(id_list)
    & data_link_full['end'].isin(id_list)
]

data_link_filtered

,start,end
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659
5,62,279122
6,62,1089199
7,62,1354553
8,62,1400636
9,62,1403619


In [42]:
data_link_grouped = data_link_filtered.groupby('start')

adiacency_list = {}

for key,item in tqdm(data_link_grouped, total=len(data_link_grouped)):
    if data_link_grouped.get_group(key)["end"].tolist()[0] == 'None':
        adiacency_list[key] = set()
    else:
        adiacency_list[key] = set(data_link_grouped.get_group(key)["end"].tolist())
    
adiacency_list



  0%|          | 0/428957 [00:00<?, ?it/s]

  0%|          | 1/428957 [00:00<100:37:40,  1.18it/s]

  0%|          | 93/428957 [00:00<70:27:53,  1.69it/s]

  0%|          | 190/428957 [00:01<49:21:04,  2.41it/s]

  0%|          | 284/428957 [00:01<34:34:36,  3.44it/s]

  0%|          | 375/428957 [00:01<24:14:17,  4.91it/s]

  0%|          | 463/428957 [00:01<17:00:15,  7.00it/s]

  0%|          | 564/428957 [00:01<11:56:08,  9.97it/s]

  0%|          | 655/428957 [00:01<8:23:34, 14.18it/s] 

  0%|          | 742/428957 [00:01<5:54:54, 20.11it/s]

  0%|          | 834/428957 [00:01<4:10:44, 28.46it/s]

  0%|          | 933/428957 [00:01<2:57:39, 40.15it/s]

  0%|          | 1038/428957 [00:01<2:06:23, 56.43it/s]

  0%|          | 1145/428957 [00:02<1:30:27, 78.82it/s]

  0%|          | 1246/428957 [00:02<1:05:26, 108.92it/s]

  0%|          | 1345/428957 [00:02<48:01, 148.39it/s]  

  0%|          | 1443/428957 [00:02<36:00, 197.87it/s]

  0%|          | 1543/428957 [00:02<27:21, 260

  3%|▎         | 14083/428957 [00:15<06:53, 1004.53it/s]

  3%|▎         | 14184/428957 [00:15<06:54, 1000.36it/s]

  3%|▎         | 14285/428957 [00:15<06:54, 1000.42it/s]

  3%|▎         | 14386/428957 [00:16<06:54, 1000.47it/s]

  3%|▎         | 14489/428957 [00:16<06:51, 1006.36it/s]

  3%|▎         | 14593/428957 [00:16<06:48, 1013.42it/s]

  3%|▎         | 14695/428957 [00:16<07:07, 969.27it/s] 

  3%|▎         | 14797/428957 [00:16<07:02, 981.27it/s]

  3%|▎         | 14901/428957 [00:16<06:55, 995.48it/s]

  3%|▎         | 15001/428957 [00:16<07:10, 962.48it/s]

  4%|▎         | 15109/428957 [00:16<06:57, 992.37it/s]

  4%|▎         | 15214/428957 [00:16<06:51, 1006.25it/s]

  4%|▎         | 15319/428957 [00:16<06:47, 1016.20it/s]

  4%|▎         | 15421/428957 [00:17<06:48, 1011.46it/s]

  4%|▎         | 15523/428957 [00:17<06:51, 1005.19it/s]

  4%|▎         | 15624/428957 [00:17<07:10, 960.82it/s] 

  4%|▎         | 15721/428957 [00:17<07:49, 879.88it/s]

  4%|▎         | 15

  7%|▋         | 27978/428957 [00:30<07:17, 917.10it/s]

  7%|▋         | 28086/428957 [00:31<06:58, 958.16it/s]

  7%|▋         | 28183/428957 [00:31<07:06, 939.49it/s]

  7%|▋         | 28285/428957 [00:31<06:57, 959.72it/s]

  7%|▋         | 28382/428957 [00:31<07:07, 937.82it/s]

  7%|▋         | 28484/428957 [00:31<06:57, 958.50it/s]

  7%|▋         | 28581/428957 [00:31<07:03, 945.22it/s]

  7%|▋         | 28676/428957 [00:31<07:12, 924.70it/s]

  7%|▋         | 28773/428957 [00:31<07:07, 935.29it/s]

  7%|▋         | 28876/428957 [00:31<06:57, 959.29it/s]

  7%|▋         | 28973/428957 [00:31<07:07, 934.82it/s]

  7%|▋         | 29068/428957 [00:32<07:06, 936.71it/s]

  7%|▋         | 29163/428957 [00:32<07:06, 938.03it/s]

  7%|▋         | 29257/428957 [00:32<07:09, 930.42it/s]

  7%|▋         | 29353/428957 [00:32<07:06, 936.51it/s]

  7%|▋         | 29452/428957 [00:32<07:00, 949.36it/s]

  7%|▋         | 29550/428957 [00:32<06:57, 955.71it/s]

  7%|▋         | 29646/428957 [

 10%|▉         | 41990/428957 [00:45<06:36, 977.08it/s]

 10%|▉         | 42091/428957 [00:45<06:33, 984.01it/s]

 10%|▉         | 42192/428957 [00:46<06:31, 988.92it/s]

 10%|▉         | 42300/428957 [00:46<06:22, 1011.90it/s]

 10%|▉         | 42402/428957 [00:46<06:23, 1008.48it/s]

 10%|▉         | 42503/428957 [00:46<06:29, 991.29it/s] 

 10%|▉         | 42604/428957 [00:46<06:28, 994.06it/s]

 10%|▉         | 42704/428957 [00:46<06:28, 993.04it/s]

 10%|▉         | 42804/428957 [00:46<06:38, 969.25it/s]

 10%|█         | 42911/428957 [00:46<06:28, 994.81it/s]

 10%|█         | 43011/428957 [00:46<06:31, 984.76it/s]

 10%|█         | 43119/428957 [00:46<06:22, 1008.85it/s]

 10%|█         | 43221/428957 [00:47<06:26, 997.50it/s] 

 10%|█         | 43326/428957 [00:47<06:21, 1009.94it/s]

 10%|█         | 43428/428957 [00:47<06:21, 1010.10it/s]

 10%|█         | 43531/428957 [00:47<06:20, 1013.16it/s]

 10%|█         | 43635/428957 [00:47<06:18, 1018.25it/s]

 10%|█         | 43737

 13%|█▎        | 56156/428957 [01:00<06:41, 927.99it/s]

 13%|█▎        | 56250/428957 [01:00<06:41, 928.96it/s]

 13%|█▎        | 56349/428957 [01:00<06:34, 943.91it/s]

 13%|█▎        | 56451/428957 [01:00<06:26, 962.95it/s]

 13%|█▎        | 56554/428957 [01:01<06:20, 979.48it/s]

 13%|█▎        | 56653/428957 [01:01<06:24, 968.38it/s]

 13%|█▎        | 56751/428957 [01:01<06:24, 969.11it/s]

 13%|█▎        | 56853/428957 [01:01<06:19, 981.16it/s]

 13%|█▎        | 56959/428957 [01:01<06:11, 1000.88it/s]

 13%|█▎        | 57060/428957 [01:01<06:17, 986.13it/s] 

 13%|█▎        | 57159/428957 [01:01<06:25, 964.39it/s]

 13%|█▎        | 57259/428957 [01:01<06:23, 969.30it/s]

 13%|█▎        | 57365/428957 [01:01<06:14, 992.20it/s]

 13%|█▎        | 57466/428957 [01:01<06:13, 994.70it/s]

 13%|█▎        | 57566/428957 [01:02<06:13, 993.48it/s]

 13%|█▎        | 57666/428957 [01:02<06:24, 966.74it/s]

 13%|█▎        | 57763/428957 [01:02<06:31, 948.03it/s]

 13%|█▎        | 57859/428957

...

 91%|█████████ | 390792/428957 [07:35<00:42, 889.39it/s]

 91%|█████████ | 390883/428957 [07:36<00:44, 852.85it/s]

 91%|█████████ | 390971/428957 [07:36<00:44, 858.44it/s]

 91%|█████████ | 391076/428957 [07:36<00:41, 905.93it/s]

 91%|█████████ | 391177/428957 [07:36<00:40, 932.39it/s]

 91%|█████████ | 391278/428957 [07:36<00:39, 951.85it/s]

 91%|█████████ | 391375/428957 [07:36<00:40, 924.55it/s]

 91%|█████████▏| 391469/428957 [07:36<00:41, 894.76it/s]

 91%|█████████▏| 391560/428957 [07:36<00:41, 896.83it/s]

 91%|█████████▏| 391662/428957 [07:36<00:40, 928.14it/s]

 91%|█████████▏| 391756/428957 [07:36<00:40, 920.88it/s]

 91%|█████████▏| 391849/428957 [07:37<00:40, 907.54it/s]

 91%|█████████▏| 391941/428957 [07:37<00:41, 882.56it/s]

 91%|█████████▏| 392030/428957 [07:37<00:41, 879.69it/s]

 91%|█████████▏| 392130/428957 [07:37<00:40, 910.28it/s]

 91%|█████████▏| 392230/428957 [07:37<00:39, 932.99it/s]

 91%|█████████▏| 392324/428957 [07:37<00:42, 870.29it/s]

 91%|█████████

 94%|█████████▎| 401443/428957 [07:51<00:32, 844.94it/s]

 94%|█████████▎| 401537/428957 [07:51<00:31, 869.10it/s]

 94%|█████████▎| 401639/428957 [07:51<00:30, 907.18it/s]

 94%|█████████▎| 401739/428957 [07:51<00:29, 930.70it/s]

 94%|█████████▎| 401840/428957 [07:51<00:28, 950.62it/s]

 94%|█████████▎| 401943/428957 [07:51<00:27, 970.53it/s]

 94%|█████████▎| 402042/428957 [07:52<00:28, 945.69it/s]

 94%|█████████▎| 402138/428957 [07:52<00:29, 899.39it/s]

 94%|█████████▍| 402229/428957 [07:52<00:32, 815.40it/s]

 94%|█████████▍| 402314/428957 [07:52<00:32, 823.21it/s]

 94%|█████████▍| 402409/428957 [07:52<00:31, 855.37it/s]

 94%|█████████▍| 402496/428957 [07:52<00:34, 762.67it/s]

 94%|█████████▍| 402576/428957 [07:52<00:37, 709.83it/s]

 94%|█████████▍| 402650/428957 [07:52<00:36, 714.57it/s]

 94%|█████████▍| 402724/428957 [07:53<00:38, 682.22it/s]

 94%|█████████▍| 402799/428957 [07:53<00:37, 699.38it/s]

 94%|█████████▍| 402871/428957 [07:53<00:37, 693.34it/s]

 94%|█████████

 97%|█████████▋| 413945/428957 [08:06<00:22, 673.77it/s]

 97%|█████████▋| 414014/428957 [08:06<00:22, 661.13it/s]

 97%|█████████▋| 414081/428957 [08:06<00:23, 646.59it/s]

 97%|█████████▋| 414147/428957 [08:07<00:24, 602.57it/s]

 97%|█████████▋| 414209/428957 [08:07<00:26, 558.55it/s]

 97%|█████████▋| 414275/428957 [08:07<00:25, 584.08it/s]

 97%|█████████▋| 414350/428957 [08:07<00:23, 621.03it/s]

 97%|█████████▋| 414414/428957 [08:07<00:24, 591.95it/s]

 97%|█████████▋| 414475/428957 [08:07<00:24, 587.01it/s]

 97%|█████████▋| 414535/428957 [08:07<00:26, 541.39it/s]

 97%|█████████▋| 414591/428957 [08:07<00:26, 534.41it/s]

 97%|█████████▋| 414646/428957 [08:07<00:27, 528.96it/s]

 97%|█████████▋| 414700/428957 [08:08<00:27, 520.00it/s]

 97%|█████████▋| 414768/428957 [08:08<00:25, 558.21it/s]

 97%|█████████▋| 414865/428957 [08:08<00:22, 638.49it/s]

 97%|█████████▋| 414955/428957 [08:08<00:20, 697.93it/s]

 97%|█████████▋| 415056/428957 [08:08<00:18, 767.58it/s]

 97%|█████████

 99%|█████████▉| 426650/428957 [08:21<00:03, 734.58it/s]

 99%|█████████▉| 426739/428957 [08:21<00:02, 773.29it/s]

100%|█████████▉| 426842/428957 [08:21<00:02, 833.87it/s]

100%|█████████▉| 426945/428957 [08:21<00:02, 882.25it/s]

100%|█████████▉| 427050/428957 [08:21<00:02, 924.36it/s]

100%|█████████▉| 427152/428957 [08:21<00:01, 948.61it/s]

100%|█████████▉| 427249/428957 [08:22<00:01, 943.94it/s]

100%|█████████▉| 427345/428957 [08:22<00:01, 946.06it/s]

100%|█████████▉| 427443/428957 [08:22<00:01, 953.36it/s]

100%|█████████▉| 427543/428957 [08:22<00:01, 964.25it/s]

100%|█████████▉| 427643/428957 [08:22<00:01, 972.03it/s]

100%|█████████▉| 427744/428957 [08:22<00:01, 980.42it/s]

100%|█████████▉| 427843/428957 [08:22<00:01, 974.74it/s]

100%|█████████▉| 427941/428957 [08:22<00:01, 973.54it/s]

100%|█████████▉| 428039/428957 [08:22<00:00, 941.92it/s]

100%|█████████▉| 428137/428957 [08:22<00:00, 950.42it/s]

100%|█████████▉| 428233/428957 [08:23<00:00, 933.96it/s]

100%|█████████

{52: {401135, 1069112, 1163551},
 62: {12162,
  167659,
  279122,
  1089199,
  1354553,
  1400636,
  1403619,
  1537692,
  1544420},
 64: {64873},
 66: {279122, 1163290},
 74: {279122},
 95: {1179478, 1185516, 1185519, 1230865},
 103: {107},
 104: {107, 1174251, 1174302},
 105: {1174967},
 107: {104, 142167, 657930, 1013995, 1061780, 1174302, 1179210, 1181401},
 108: {104, 107, 1059989, 1062426, 1161925, 1169534},
 112: {107},
 113: {279122, 1185127},
 122: {1174967},
 126: {279122},
 133: {279122, 825464},
 134: {541222, 1060414, 1061485, 1062226, 1163610, 1163783, 1265505},
 137: {136, 1163286, 1163712, 1224802},
 153: {154,
  260842,
  725249,
  744272,
  1064807,
  1065282,
  1065320,
  1066493,
  1085943,
  1125916,
  1130618,
  1161083,
  1162104,
  1162238,
  1162816,
  1532744},
 154: {153, 279122, 673672, 1057077, 1063928, 1067637, 1175075},
 155: {279122, 1539938},
 156: {279122, 785921},
 158: {279122, 1262459, 1400635, 1403118},
 160: {598053, 1108161},
 163: {896828},
 166

In [20]:
breadth_first_search(adiacency_list,64)


  0%|          | 14/461193 [00:01<15:51:49,  8.08it/s]
11652378it [11:50, 36700.07it/s]
  0%|          | 74/461193 [00:06<9:51:28, 12.99it/s]

KeyboardInterrupt: 

We can now calculate the distance of two categories.

Let's select 'English Footballers" as C0.

In [24]:
c0_articles = list(set(df_full['List of articles'][0]))

c1_articles = list(set(df_full['List of articles'][1]) - set(c0_articles))

In [ ]:
for c0_article in tqdm(c0_articles, total=len(c0_articles),desc='C0 loop', position=0):
    breadth_first_search(adiacency_list,c0_article)

Starting node: 622642:   0%|          | 366/428957 [00:27<5:24:26, 22.02it/s]


Starting node: 622642:   0%|          | 1010/428957 [00:49<4:05:19, 29.07it/s]


Starting node: 622642:   0%|          | 1212/428957 [01:29<8:31:59, 13.92it/s] 


Starting node: 622642:   0%|          | 1938/428957 [01:50<4:34:18, 25.94it/s]


Starting node: 622642:   1%|          | 2513/428957 [02:21<10:12:43, 11.60it/s]


...

Starting node: 622642:   4%|▎         | 15423/428957 [10:46<7:04:24, 16.24it/s]


Starting node: 622642:   4%|▎         | 15875/428957 [11:06<7:33:46, 15.17it/s]


Starting node: 622642:   4%|▍         | 16189/428957 [11:24<8:12:41, 13.96it/s]


Starting node: 622642:   4%|▍         | 17035/428957 [11:54<6:03:58, 18.86it/s]


Starting node: 622642:   4%|▍         | 17158/428957 [12:02<7:29:21, 15.27it/s]